In [12]:
import glob
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("../config/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# GCP 클라이언트 객체 생성
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)

In [10]:
# 데이터 조회 쿼리
sql = f"""
SELECT *
FROM `data-analysis-413914.kobis.daily_box_office`
"""

# 데이터 조회 쿼리 실행 결과
query_job = client.query(sql)

# 데이터프레임 변환
df = query_job.to_dataframe()
df.head()

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,targetDt
0,1,1,0,NEW,20226431,오펜하이머,2023-08-15,5763388463,44.2,5763388463,100.0,5773996463,552990,552990,100.0,553614,1607,5433,2023-08-15
1,3,3,0,NEW,20226500,달짝지근해: 7510,2023-08-15,1198935925,9.2,1198935925,100.0,1292425925,122202,122202,100.0,132654,779,2226,2023-08-15
2,7,7,0,NEW,20192995,보호자,2023-08-15,409485801,3.1,409485801,100.0,447967801,40221,40221,100.0,44111,685,1557,2023-08-15
3,10,10,1,OLD,20232618,런닝맨: 리벤져스,2023-08-10,69625700,0.5,45613800,190.0,305855400,7890,5102,183.0,36186,186,206,2023-08-15
4,8,8,2,OLD,20232698,바다 탐험대 옥토넛 어보브 앤 비욘드 : 육지 넘어 하늘까지!,2023-08-10,119550850,0.9,93708700,362.6,493865250,12958,9944,329.9,55624,216,322,2023-08-15


### 테이블 채우기

In [53]:
from pyarrow import csv

In [68]:
data_paths = glob.glob("../data/kobis/DailyBoxOffice/*.csv")

# 테이블 ID
table_id = "data-analysis-413914.kobis.daily_box_office"
# 테이블 객체 생성
table = client.get_table(table_id)

for data_path in data_paths:
    df = csv.read_csv(data_path).to_pandas()
    df = df.astype({'movieCd':'string'})
    # 데이터프레임을 테이블에 삽입
    client.load_table_from_dataframe(df, table)

In [69]:
data_paths = glob.glob("../data/kobis/MovieList/*.csv")

# 테이블 ID
table_id = "data-analysis-413914.kobis.movie_list"
# 테이블 객체 생성
table = client.get_table(table_id)

for data_path in data_paths:
    df = csv.read_csv(data_path).to_pandas()
    df = df.astype({'movieCd':'string'})
    # 데이터프레임을 테이블에 삽입
    client.load_table_from_dataframe(df, table)